# Introduction


Since Jan. 1, 2015, [*The Washington Post*](https://www.washingtonpost.com/) has maintained **Fatal Force**, a comprehensive record of every on-duty police shooting in the United States. To date, the database chronicles over **10,400** individual cases—each enriched with dozens of attributes, including:

- **Race**, **age**, and **gender** of the person killed  
- Whether the victim was **armed**  
- Whether the victim was in a **mental-health crisis**  
- The **agency** involved  

<center><img src="https://i.imgur.com/sX3K62b.png"></center>
<figcaption><small>Figure 1. Monthly counts of civilians shot and killed by on-duty U.S. police officers (Jan 2015–Dec 2024).</small></figcaption>

---

## Purpose of this Notebook: Census API Integration

The goal of this notebook is to **pull socioeconomic indicators directly from the U.S. Census Bureau API** and align them with the Fatal Force database. By automating this retrieval, we ensure results are **up-to-date, reproducible, and standardized** for further analysis.  

**ACS 2023 5-year variables accessed via the Census API:**

| Theme      | Description                  | API Variable(s) |
|------------|------------------------------|-----------------|
| Income     | Median household income      | `B19013_001E`   |
| Poverty    | % below poverty line         | `B17001_002E` / `B17001_001E` |
| Education  | % ≥ high-school diploma      | `B15003_017E–B15003_025E` / `B15003_001E` |
| Race       | Racial composition shares    | `B02001_00xE`   |

---

## Workflow in this Notebook

1. **Query the Census API** for each ACS variable  
2. **Collect JSON responses** and convert them into structured DataFrames  
3. **Standardize GEOIDs** to ensure compatibility with Fatal Force city/state keys  
4. **Merge** socioeconomic attributes into a single analysis-ready dataset  
5. **Validate** completeness (missing values, duplicates)  
6. **Export** the clean dataset for downstream notebooks (EDA, ML models, dashboards)  

**Data sources:**  
- Fatal Force (The Washington Post)  
- ACS 2023 5-year estimates via [data.census.gov](https://data.census.gov/) and the U.S. Census API  

---

> **Note:** This notebook is dedicated exclusively to **API data collection and preprocessing**. Modeling and visualization are handled in separate notebooks.



### Install Required Package

To access the U.S. Census Bureau’s API data programmatically, we use the **`censusdata`** Python package.  
This library simplifies the process of querying the **American Community Survey (ACS)** and other Census datasets, and returns results in a pandas-friendly format.


In [1]:
%pip install censusdata

Note: you may need to restart the kernel to use updated packages.


## Import Statements

## Step 2: Import Required Libraries

We begin by importing the core Python libraries that will support data collection, processing, and visualization:

- **numpy (`np`)** → numerical computations and array operations  
- **pandas (`pd`)** → data manipulation and tabular structures (DataFrames)  
- **plotly.express (`px`)** → interactive data visualization  
- **matplotlib.pyplot (`plt`)** → static plots and charts  
- **seaborn (`sns`)** → enhanced statistical visualizations  
- **censusdata** → direct access to Census Bureau datasets (ACS, Decennial Census)  
- **pathlib (`Path`)** → filesystem navigation and path handling  

Additional utility:  
- **collections.Counter** → (optional) quick frequency counts for categorical data

✅ With these imports, the environment is ready for Census API queries, data wrangling, and visualization.


In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import censusdata
from pathlib import Path



# This might be helpful:
from collections import Counter

## Notebook Presentation Settings & API Key

1. **Number Formatting**


In [4]:
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
import os
# Paste your key here—**don’t** check this cell into a public repo!
os.environ["CENSUS_API_KEY"] = "80758afe30fd12f3a63654f2b8b5ec2386964d7e"


##  Fetching Place-Level ACS Data via Census API

- **Endpoint**: `https://api.census.gov/data/2023/acs/acs5`  
- **API Key**: pulled from `CENSUS_API_KEY`  
- **FIPS2USPS**: maps state FIPS → USPS codes  
- **`fetch()`**: queries ACS vars, parses JSON → DataFrame with city names & codes  

**Example**  
```python
df_income = fetch("B19013_001E")   # Median Household Income
df_poverty = fetch("B17001_002E,B17001_001E")  # Poverty vars



In [6]:
import os, requests, pandas as pd

API_KEY = os.getenv("CENSUS_API_KEY")
if not API_KEY:
    raise RuntimeError("Please `export CENSUS_API_KEY=…` first")

BASE       = "https://api.census.gov/data/2023/acs/acs5"
FIPS2USPS  = {
 '01':'AL','02':'AK','04':'AZ','05':'AR','06':'CA','08':'CO','09':'CT',
 '10':'DE','11':'DC','12':'FL','13':'GA','15':'HI','16':'ID','17':'IL',
 '18':'IN','19':'IA','20':'KS','21':'KY','22':'LA','23':'ME','24':'MD',
 '25':'MA','26':'MI','27':'MN','28':'MS','29':'MO','30':'MT','31':'NE',
 '32':'NV','33':'NH','34':'NJ','35':'NM','36':'NY','37':'NC','38':'ND',
 '39':'OH','40':'OK','41':'OR','42':'PA','44':'RI','45':'SC','46':'SD',
 '47':'TN','48':'TX','49':'UT','50':'VT','51':'VA','53':'WA','54':'WV',
 '55':'WI','56':'WY'
}

def fetch(vars_list):
    """Fetch NAME plus vars_list (comma-string) for all places in all states."""
    params = {
      "get": "NAME," + vars_list,
      "for": "place:*",
      "in":  "state:*",
      "key": API_KEY
    }
    r = requests.get(BASE, params=params)
    r.raise_for_status()
    data = r.json()
    return pd.DataFrame(data[1:], columns=data[0])


##  Fetching & Cleaning Median Household Income (ACS 2023)

- Variable: `B19013_001E` → Median Household Income (USD)  
- Clean steps: convert to numeric, replace Census nulls, zero-pad FIPS, strip city names  
- Add `"Geographic Area"` from state FIPS → USPS code  
- Exported as **`Median_Household_Income_2023.csv`**


In [7]:
# fetch & clean
df = fetch("B19013_001E").assign(
    median_income=lambda d: pd.to_numeric(d.B19013_001E, errors="coerce")
                            .replace(-666666666, pd.NA),
    state=lambda d: d.state.str.zfill(2),
    City=lambda d: d.NAME.str.replace(r",.*", "", regex=True),
)
df["Geographic Area"] = df.state.map(FIPS2USPS)

# select & write
(
    df[["Geographic Area","City","median_income"]]
      .rename(columns={"median_income":"Median Income"})
      .to_csv("Median_Household_Income_2023.csv", index=False)
)
print("✅ Median_Household_Income_2023.csv written")


✅ Median_Household_Income_2023.csv written


##  Fetching & Cleaning High-School Completion Rate (ACS 2023)

- Variables: `B15003_017E–025E` (HS+ diploma bins), denominator `B15003_001E` (total pop ≥25)  
- Steps: fetch → convert to numeric → compute `% HS+` = (sum of HS bins ÷ total) × 100  
- Format: clean city names, pad FIPS, map to USPS state codes  



## 2) Fetch High-school completion rate

In [8]:
# codes for B15003_017E…025E (HS grad bins) + total B15003_001E
hs_codes = [f"B15003_{i:03d}E" for i in range(17,26)]
total_code = "B15003_001E"

# fetch NAME + hs_codes + total_code
df = fetch(",".join(hs_codes + [total_code]))

# convert only those ACS columns to numeric
df[hs_codes + [total_code]] = df[hs_codes + [total_code]].apply(
    pd.to_numeric, errors="coerce"
)

# compute % ≥HS
df["pct_completed_hs"] = (
    df[hs_codes].sum(axis=1) 
    / df[total_code] * 100
)

# format
df["state"] = df.state.str.zfill(2)
df["City"]  = df.NAME.str.replace(r",.*", "", regex=True)
df["Geographic Area"] = df.state.map(FIPS2USPS)

# select & write
(
    df[["Geographic Area","City","pct_completed_hs"]]
      .rename(columns={"pct_completed_hs":"percent_completed_hs"})
      .round({"percent_completed_hs":1})
      .to_csv("Pct_Completed_HS_2023.csv", index=False)
)
print("✅ Pct_Completed_HS_2023.csv written")




✅ Pct_Completed_HS_2023.csv written


##  Fetching & Cleaning Poverty Rate (ACS 2023)

- Variables: `B17001_002E` (below poverty) ÷ `B17001_001E` (total pop)  
- Steps: fetch → convert numeric → compute `% poverty`  
- Format: pad FIPS, clean city names, map USPS state codes  



## 3) Fetch poverty rate

In [9]:
# below‐poverty / total‐pop codes
below_code = "B17001_002E"
total_pop  = "B17001_001E"

# fetch just those two
df = fetch(f"{below_code},{total_pop}")

# convert them
df[[below_code, total_pop]] = df[[below_code, total_pop]].apply(
    pd.to_numeric, errors="coerce"
)

# compute %
df["poverty_rate"] = df[below_code] / df[total_pop] * 100

# format
df["state"] = df.state.str.zfill(2)
df["City"]  = df.NAME.str.replace(r",.*", "", regex=True)
df["Geographic Area"] = df.state.map(FIPS2USPS)

# select & write
(
    df[["Geographic Area","City","poverty_rate"]]
      .round({"poverty_rate":1})
      .to_csv("Pct_People_Below_Poverty_Level_2023.csv", index=False)
)
print("✅ Pct_People_Below_Poverty_Level_2023.csv written")




✅ Pct_People_Below_Poverty_Level_2023.csv written


##  Fetching & Cleaning Race Composition (ACS 2023)

- Variables: `B02001_002E–008E` → race groups (White, Black, Native, Asian, Pacific Islander, Other, Two+)  
- Steps: fetch → convert numeric → compute % shares per group  
- Format: pad FIPS, clean city names, map USPS state codes  



## 4) Fetch race shares

In [10]:
# map of share‐fields
race_codes = {
  "share_white":  "B02001_002E",
  "share_black":  "B02001_003E",
  "share_native": "B02001_004E",
  "share_asian":  "B02001_005E",
  "share_pacisl": "B02001_006E",
  "share_other":  "B02001_007E",
  "share_two+":   "B02001_008E"
}

# fetch only those
df = fetch(",".join(race_codes.values()))

# convert just them
df[list(race_codes.values())] = df[list(race_codes.values())].apply(
    pd.to_numeric, errors="coerce"
)

# compute totals & shares
tot = df[list(race_codes.values())].sum(axis=1)
for name, code in race_codes.items():
    df[name] = df[code] / tot * 100

# format
df["state"] = df.state.str.zfill(2)
df["City"]  = df.NAME.str.replace(r",.*", "", regex=True)
df["Geographic Area"] = df.state.map(FIPS2USPS)

# select & write
out = df[["Geographic Area","City"] + list(race_codes.keys())].round(1)
out.to_csv("Share_of_Race_By_City_2023.csv", index=False)
print("✅ Share_of_Race_By_City_2023.csv written")



✅ Share_of_Race_By_City_2023.csv written
